This notebook demonstrates how to calculate and plot eFold distance of inter-site temperature correlation using the eFold module. The source data for this example is NOAA's Global Historical Climate Network Monthly temperature dataset (https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-monthly-version-4).

The source code in the notebook converts the fixed width ASCII dataset into a set of times and temperatures, then uses the eFold module to filter the dataset, calculate the correlations between sites, then calculate and plot the eFold distances between the correlations.

Working with larger datasets on memory- or CPU-limited computers can be slow. There are several checkpoints in the notebook where processed data can be saved or restored via a pickle file, to reduce times of future runs.

In [ ]:
#import system packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy
import sys
import io
import warnings
import pickle


In [ ]:
#import local modules
from eFold import binTime
from eFold import bandPassFilter
from eFold import binAndFilter
from eFold import eFoldingDistance
from eFold import calcEFold
from eFold import plotMap


In [ ]:
# Specify input file parameters
monthNames = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
inputDataFid = 'ghcnm.tavg.v3.3.0.20171130.qca.dat'
inputMetafileFid = 'ghcnm.tavg.v3.3.0.20171130.qca.inv'
inputFirstLastFid = 'ghcnm.v3.first.last.txt'
dataTColumns = ['id','year','element','jan','dm1','qc1','ds1','feb','dm2','qc2','ds2','mar','dm3','qc3','ds3',
                'apr','dm4','qc4','ds4','may','dm5','qc5','ds5','jun','dm6','qc6','ds6','jul','dm7','qc7','ds7',
                'aug','dm8','qc8','ds8','sep','dm9','qc9','ds9','oct','dm10','qc10','ds10',
                'nov','dm11','qc11','ds11','dec','dm12','qc12','ds12' ]
dataTWidths = [ 11,4,4,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1,5,1,1,1 ]
minimumDuration = 40

In [ ]:
# Load temperatures from input file into dataframe
dataT = pd.read_fwf(inputDataFid,widths=dataTWidths,names=dataTColumns)

# In this dataset, missing readings are represented by the value -9999,
# so change all -9999 to NaN
dataT = dataT.replace(-9999,np.NaN)
dataT.shape

In [ ]:
# Load Metadata from input file into dataframe
metadataColumns = ['id','latitude','longitude','elevation','sitename',
                 'GRELEV','POPCLS','POPSIZ','TOPO','STVEG','STLOC',
                 'OCNDIS','AIRSTN','TOWNDIS','GRVEG','POPCSS']
metadataWidths = [11,9,10,7,31,5,1,5,2,2,2,2,1,2,16,1]
metadata = pd.read_fwf(inputMetafileFid,widths=metadataWidths,names=metadataColumns)
metadata.shape


In [ ]:
# Load the FirstLast data into a dataframe
firstLastColumnNames = ['id','duration']
firstLastColspecs = [(0,11),(61,65)]
firstLast = pd.read_fwf(inputFirstLastFid,colspecs=firstLastColspecs,names=firstLastColumnNames,skiprows=3)


In [ ]:
# Merge the duration from the FirstLast DF into the metadata DF
metadata = pd.merge(metadata,firstLast,on='id',how='left')

# Filter out short durations
goodMetadata = metadata[metadata.duration.ge(minimumDuration)]
goodMetadata.shape

In [ ]:
#
# Create a list containing one dictionary for each site
# Each site dictionary will contain metadata:latitude, longitude, elevation, sitename, and oceanDist, 
# along with a dataframe of monthly temperatures
#
siteList = []


# FOR TESTING USING A SMALLER DATASET, modify this values to use only every nth datasite.
useNthSite = 1

#iterate over all the sites
for index,site in goodMetadata.iterrows():
    if index % useNthSite != 0:
        continue
    siteDict = {}
    siteDict['latitude'] = site['latitude']
    siteDict['longitude'] = site['longitude']
    siteDict['sitename'] = site['sitename']

    #select the temperature data for this site only
    siteT = dataT[dataT['id']==site['id']]

    # Pull out all the monthly temperatures, and divide by 100 (convert units from millidegrees to degrees)
    monthlyT = siteT[monthNames]/100.0

    # Add the years field into the dataframe
    years = siteT['year']

    # Create array of monthly differences from the mean
    valsArray = np.array(monthlyT.loc[:,'jan':'dec'])

    # Create matching array of the year/month for each value
    months = np.tile(np.arange(1/24,1.0,1/12),(len(years),1))
    years = np.tile(years,(12,1)).transpose()
    timeArray = months+years

    siteDict['temps'] = np.array(monthlyT).flatten()
    siteDict['times'] = timeArray.flatten()

    siteList.append(siteDict)

len(siteList)


In [ ]:
# Uncomment the following to save the siteList for later

#outPickleFid='ipynb.sitelist.GHCN.pickle.dat'
#pickle.dump(siteList, open(outPickleFid,'wb'))

# If running with previously loaded data, uncomment and run:

#pickleFidIn = 'ipynb.sitelist.GHCN.pickle.dat'
#siteList = pickle.load(open(pickleFidIn,'rb'))

In [ ]:
# bin all the temperatures into years, and filter out the low frequency repetitions
pickleFidOut = 'ipynb.filtered.GHCN.pickle.dat'
filteredArray = binAndFilter( siteList, timeStart=1900,timeEnd=2011,timeStep=1,highBandPass=True,replaceNaN=True,replaceNaNDivisor=12,outFid=pickleFidOut )

In [ ]:
# Calculate correlations between each pair of sites and the eFold distance for each site
eFold = calcEFold( siteList, filteredArray )

In [ ]:

# Plot the data on a map
# Put the data into usable lists                     
lons = []                                            
lats = []                                            
r2 = []                                              
eFoldDistance = []                                   
for i in range(0,len(eFold)):                        
    lons.append(eFold[i]['lon'])                     
    lats.append(eFold[i]['lat'])
    r2.append(eFold[i]['r2'])
    eFoldDistance.append(eFold[i]['eFoldDistance'])

plt.rcParams['figure.figsize']=[10,5]
plotMap( lats, lons, eFoldDistance,plotTitle='GHCN40',dataLabel='e-Folding Distance (km)' )
plotMap( lats, lons,r2,plotTitle='GHCN40',dataLabel='$r^2$ for eFoldDistance',dotSize=10)  
